# Load-Model FineTuning


In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 84.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.3 MB/s

In [5]:
!pip install transformers langchain langchain-community torch pypdf sentence-transformers gpt4all faiss-cpu openai ctransformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
 

# Llama3.2


In [13]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
import torch
import logging
import contextlib
import os

# Tắt logging từ thư viện Unsloth
logging.getLogger("unsloth").setLevel(logging.ERROR)

class LanguageModelHandler:
    def __init__(self, model_path, max_seq_length=2048, load_in_4bit=True):
        self.max_seq_length = max_seq_length
        self.load_in_4bit = load_in_4bit
        
        # Redirect stdout để ẩn thông báo
        with open(os.devnull, "w") as devnull:
            with contextlib.redirect_stdout(devnull):
                self.model, self.tokenizer = self.load_model(model_path)

    def load_model(self, model_path):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_path,
            max_seq_length=self.max_seq_length,
            load_in_4bit=self.load_in_4bit
        )
        
        # Gọi phương thức for_inference
        FastLanguageModel.for_inference(model)

        tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
        return model, tokenizer
    
    def generate_response(self, messages, max_new_tokens=512, temperature=0.1, min_p=0.1):
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")
        
        outputs = self.model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            min_p=min_p
        )
        
        # Lấy kết quả và thêm định dạng yêu cầu
        result = self.tokenizer.batch_decode(outputs)
        formatted_result = f"{result[0].strip()} <|eot_id|>"
        return formatted_result
    
    def stream_response(self, messages, max_new_tokens=512, temperature=0.1, min_p=0.1):
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")
        
        text_streamer = TextStreamer(self.tokenizer, skip_prompt=True)
        self.model.generate(
            input_ids=inputs,
            streamer=text_streamer,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            min_p=min_p
        )

# Tách hàm nhập câu hỏi
def get_user_input():
    return input("Nhập câu hỏi của bạn (hoặc nhập 'exit' để thoát): ").strip()

# Tách hàm trả kết quả từ mô hình
def get_model_response(model_handler, user_question):
    messages = [
        {"role": "user", "content": user_question},
    ]
    response = model_handler.generate_response(messages)
    print(response)

# Sử dụng lớp LanguageModelHandler với đường dẫn tới mô hình
model_handler = LanguageModelHandler("/kaggle/input/llm-llama3-2/Model")

# Vòng lặp cho phép người dùng hỏi nhiều lần
while True:
    user_question = get_user_input()

    if user_question.lower() == 'exit':
        print("Thoát chương trình.")
        break

    get_model_response(model_handler, user_question)

# Nếu bạn muốn sử dụng streaming
# model_handler.stream_response(messages)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Nhập câu hỏi của bạn (hoặc nhập 'exit' để thoát):  Vì sao 1 + 1 bằng 2


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Vì sao 1 + 1 bằng 2<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Câu hỏi này có vẻ như đang đề cập đến một vấn đề cơ bản về toán học, nhưng có thể có một số hiểu lầm hoặc ngữ cảnh cụ thể mà tôi cần biết để trả lời chính xác.

Trong toán học, 1 + 1 thực sự bằng 2 theo định nghĩa cơ bản của toán học. Đây là một trong những chân lý cơ bản nhất trong toán học, và nó được chấp nhận rộng rãi trên toàn thế giới.

Tuy nhiên, có thể bạn đang đề cập đến một vấn đề cụ thể trong một lĩnh vực toán học nào đó, chẳng hạn như:

* Trong số học, 1 + 1 có thể được biểu diễn dưới dạng các số nguyên hoặc các số thực, và kết quả có thể khác nhau tùy thuộc vào hệ thống số mà bạn đang sử dụng.
* Trong đại số, 1 + 1 có thể được biểu diễn dưới dạng các phương trình hoặc các hàm toán học, và kết quả có thể khác nhau tùy thuộc v

Nhập câu hỏi của bạn (hoặc nhập 'exit' để thoát):  exit


Thoát chương trình.


# Llama3.1


In [ ]:
from unsloth import FastLanguageModel
import torch

class LanguageModelInference:
    def __init__(self, model_name, max_seq_length=2048, dtype=None, load_in_4bit=True):
        self.model_name = model_name
        self.max_seq_length = max_seq_length
        self.dtype = dtype
        self.load_in_4bit = load_in_4bit
        self.model, self.tokenizer = self.load_model()
        self.prepare_model_for_inference()

    def load_model(self):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=self.model_name,
            max_seq_length=self.max_seq_length,
            dtype=self.dtype,
            load_in_4bit=self.load_in_4bit
        )
        return model, tokenizer

    def prepare_model_for_inference(self):
        FastLanguageModel.for_inference(self.model)

    def generate_response(self, instruction, input_text="", max_new_tokens=128):
        alpaca_prompt = """Tôi là MeGPT, sẵn sàng trả lời câu hỏi của bạn, nhưng chỉ với thông tin từ tháng 5/2024 trở về trước.

        ### Instruction:
        {}

        ### Input:
        {}

        ### Response:
        {}"""

        prompt = alpaca_prompt.format(instruction, input_text, "")
        inputs = self.tokenizer(
            [prompt + self.tokenizer.eos_token],
            return_tensors="pt"
        ).to("cuda")

        outputs = self.model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
        response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return response[0]

model_name = "/kaggle/input/llm-llama3-1/content/lora_model"
inference_model = LanguageModelInference(model_name)

instruction = "Tam giác nào có ba góc bằng 60 độ?"
response = inference_model.generate_response(instruction)
print("Câu trả lời:", response)


# RAG


In [16]:
#Sử dụng cho pdf
import os
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Khai báo biến
pdf_data_path = "/kaggle/input/llm-langchain-test/data"
vector_db_path = "vectorstores/db_faiss"

if not os.path.exists(pdf_data_path):
    print(f"Thư mục '{pdf_data_path}' does not exist. Creating new...")
    os.makedirs(pdf_data_path)

if os.path.exists(vector_db_path):
    shutil.rmtree(vector_db_path)

# Tạo thư mục cho vector DB
os.makedirs(vector_db_path)

# Hàm tạo vector DB từ các tệp PDF
def create_db_from_files():
    loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    if not documents:
        print("Không tìm thấy tài liệu nào trong thư mục PDF.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)

    if not chunks:
        print("Không thể chia nhỏ tài liệu thành các đoạn. Vui lòng kiểm tra tài liệu PDF.")
        return None

    # In kết quả sau khi chia nhỏ
    print("Các đoạn văn bản đã được chia nhỏ từ file PDF:")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i + 1}:")
        print(chunk)
        print("-" * 50)

    # Embedding
    embedding_model = GPT4AllEmbeddings(model_file="/kaggle/input/llm-langchain-test/models/all-MiniLM-L6-v2-f16.gguf")
    db = FAISS.from_documents(chunks, embedding_model)
    db.save_local(vector_db_path)
    return db

# Gọi hàm để tạo vector DB từ các tệp PDF
db = create_db_from_files()


Các đoạn văn bản đã được chia nhỏ từ file PDF:
Chunk 1:
page_content='1 
 HƯỚNG D ẪN HỌC VỤ CHO TÂN SINH VIÊN TRÚNG TUY ỂN NĂM 2024 (K30) 
1. Khung th ời gian h ọc tập trong m ột ngày 
Buổi học Ca học Tiết học Thời gian  Ghi ch ú 
Sáng 1 1 (07g00  - 07g45)  
Thời khóa bi ểu của các l ớp 
học phần được bố trí đều 
trong các tuần của học kỳ, 
và tổ chức theo K ế hoạch 
tổ chức đào tạo được ban 
hành hàng năm.  
Năm h ọc 202 4-2025: Kế 
hoạch khung t ổ chức đào 
tạo năm h ọc (Academic 
Calender)  số 729/KH-
ĐHVL -ĐT ban h ành ng ày' metadata={'source': '/kaggle/input/llm-langchain-test/data/HuongDanHocVu_SV_K30.pdf', 'page': 0}
--------------------------------------------------
Chunk 2:
page_content='Calender)  số 729/KH-
ĐHVL -ĐT ban h ành ng ày 
30/06/2024. 2 (07g50  - 08g35)  
3 (08g40  - 09g25)  
2 4 (09g35  - 10g20)  
5 (10g25  - 11g10)  
6 (11g15  - 12g00)  
Chiều 3 7 (13g00  - 13g45)  
8 (13g50  - 14g35)  
9 (14g40  - 15g25)  
4 10 (15g35  - 1

In [17]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Cau hinh
model_file = "/kaggle/input/llm-langchain-test/models/vinallama-7b-chat_q5_0.gguf"
vector_db_path = "/kaggle/working/vectorstores/db_faiss"

# Load LLM
def load_llm(model_file):
    llm = CTransformers(
        model=model_file,
        model_type="llama",
        max_new_tokens=128,
        temperature=0.1
    )
    return llm

# Tao prompt template
def create_prompt(template):
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    return prompt

# Tao simple chain
def create_qa_chain(prompt, llm, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 2}, max_tokens_limit=218),
        return_source_documents=False,
        chain_type_kwargs={'prompt': prompt}
    )
    return llm_chain

# Read tu VectorDB
def read_vectors_db():
    embedding_model = GPT4AllEmbeddings(model_file="/kaggle/input/llm-langchain-test/models/all-MiniLM-L6-v2-f16.gguf")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    return db

# Bat dau thu nghiem
db = read_vectors_db()
llm = load_llm(model_file)

# Tao Prompt
template = """<|im_start|>system\nSử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, 
đừng cố tạo ra câu trả lời\n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""
prompt = create_prompt(template)

llm_chain = create_qa_chain(prompt, llm, db)

# Chay cai chain
question = "Theo thời gian học tập chính thức của học kỳ 1 năm học 2024 -2025 sẽ bắt đầu khi nào?"
response = llm_chain.invoke({"query": question})
print(response)


{'query': 'Theo thời gian học tập chính thức của học kỳ 1 năm học 2024 -2025 sẽ bắt đầu khi nào?', 'result': '\nHọc kỳ 1 năm học 2024 -2025 sẽ bắt đầu vào tuần 4 (từ ngày 23 tháng 9 năm 2024).<|im_end|>'}


# FastAPI Combined with Model-Llama3.2


In [2]:
!pip install flask torch fastapi pyngrok unsloth nest_asyncio

In [11]:
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import logging
import contextlib
import os
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
import torch
import nest_asyncio  # Import nest_asyncio

# Apply nest_asyncio to handle running in a notebook
nest_asyncio.apply()

# Tắt logging từ thư viện Unsloth
logging.getLogger("unsloth").setLevel(logging.ERROR)

# Định nghĩa lớp LanguageModelHandler
class LanguageModelHandler:
    def __init__(self, model_path, max_seq_length=2048, load_in_4bit=True):
        self.max_seq_length = max_seq_length
        self.load_in_4bit = load_in_4bit

        # Redirect stdout để ẩn thông báo
        with open(os.devnull, "w") as devnull:
            with contextlib.redirect_stdout(devnull):
                self.model, self.tokenizer = self.load_model(model_path)

    def load_model(self, model_path):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_path,
            max_seq_length=self.max_seq_length,
            load_in_4bit=self.load_in_4bit
        )
        
        # Gọi phương thức for_inference
        FastLanguageModel.for_inference(model)

        tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
        return model, tokenizer

    def generate_response(self, messages, max_new_tokens=512, temperature=0.1, min_p=0.1):
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        outputs = self.model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            min_p=min_p
        )
        
        # Lấy kết quả và thêm định dạng yêu cầu
        result = self.tokenizer.batch_decode(outputs)
        formatted_result = f"{result[0].strip()} <|eot_id|>"
        return formatted_result

    def stream_response(self, messages, max_new_tokens=512, temperature=0.1, min_p=0.1):
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")
        
        text_streamer = TextStreamer(self.tokenizer, skip_prompt=True)
        self.model.generate(
            input_ids=inputs,
            streamer=text_streamer,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            min_p=min_p
        )


# Tạo đối tượng model_handler
model_handler = LanguageModelHandler("/kaggle/input/llm-llama3-2/Model")

# Tạo FastAPI app
app = FastAPI()

# Mô hình dữ liệu cho câu hỏi từ người dùng
class Question(BaseModel):
    question: str

# Route để kiểm tra API hoạt động
@app.get("/")
def read_root():
    return {"message": "API is working"}

# Route để nhận câu hỏi và trả về câu trả lời từ mô hình
@app.post("/ask/")
def ask_question(question: Question):
    messages = [
        {"role": "user", "content": question.question},
    ]
    response = model_handler.generate_response(messages)
    return {"response": response}


ngrok.set_auth_token("2muEgcLI72W3vl8i28eJE5rrSSP_GJQ9nDAa4eQgCqZkCQ2f")

# Khởi tạo ngrok tunnel
public_url = ngrok.connect(8000)
print("Ngrok Tunnel URL:", public_url)

# Chạy server FastAPI
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Ngrok Tunnel URL: NgrokTunnel: "https://efef-34-34-58-188.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     54.86.50.139:0 - "GET / HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /ask/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [30]


KeyboardInterrupt: 

# FastAPI combined with HMTL

In [2]:
!pip install fastapi uvicorn pyngrok transformers nest_asyncio

In [14]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import contextlib
import os
import torch
from transformers import TextStreamer
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import logging

# Tắt logging từ Unsloth
logging.getLogger("unsloth").setLevel(logging.ERROR)

# Áp dụng nest_asyncio để khắc phục xung đột asyncio trong notebook
nest_asyncio.apply()

# Định nghĩa lớp LanguageModelHandler
class LanguageModelHandler:
    def __init__(self, model_path, max_seq_length=2048, load_in_4bit=True):
        self.max_seq_length = max_seq_length
        self.load_in_4bit = load_in_4bit

        # Redirect stdout để ẩn thông báo
        with open(os.devnull, "w") as devnull:
            with contextlib.redirect_stdout(devnull):
                self.model, self.tokenizer = self.load_model(model_path)

    def load_model(self, model_path):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_path,
            max_seq_length=self.max_seq_length,
            load_in_4bit=self.load_in_4bit
        )
        
        # Gọi phương thức for_inference
        FastLanguageModel.for_inference(model)

        tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
        return model, tokenizer

    def generate_response(self, messages, max_new_tokens=512, temperature=0.1, min_p=0.1):
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        outputs = self.model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            min_p=min_p
        )
        
        result = self.tokenizer.batch_decode(outputs)
        formatted_result = f"{result[0].strip()} <|eot_id|>"
        return formatted_result

# Tạo đối tượng model_handler
model_handler = LanguageModelHandler("/kaggle/input/llm-llama3-2/Model")

# Khởi tạo FastAPI app
app = FastAPI()

# Mô hình dữ liệu cho câu hỏi từ người dùng
class Question(BaseModel):
    question: str

# Route để kiểm tra API hoạt động và trả về trang HTML
@app.get("/", response_class=HTMLResponse)
def read_root():
    # Tạo một file HTML đơn giản
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Chatbot</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                margin: 0;
                padding: 20px;
            }
            #chat-container {
                border: 1px solid #ccc;
                border-radius: 5px;
                padding: 10px;
                background-color: #fff;
                max-width: 600px;
                margin: 0 auto;
                height: 400px;
                overflow-y: auto;
            }
            #user-input {
                width: 100%;
                padding: 10px;
                box-sizing: border-box;
            }
            .message {
                margin: 10px 0;
            }
            .user-message {
                text-align: right;
                color: blue;
            }
            .bot-message {
                text-align: left;
                color: green;
            }
        </style>
    </head>
    <body>

        <h1>Chatbot</h1>
        <div id="chat-container"></div>
        <input type="text" id="user-input" placeholder="Nhập câu hỏi của bạn..."/>
        <button id="send-button">Gửi</button>

        <script>
            document.getElementById('send-button').addEventListener('click', async () => {
                const userInput = document.getElementById('user-input').value;
                if (!userInput) return;

                // Hiển thị câu hỏi của người dùng
                displayMessage(userInput, 'user-message');

                // Gửi yêu cầu đến API
                const response = await fetch('/ask/', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ question: userInput }),
                });

                const data = await response.json();

                // Hiển thị câu trả lời của bot
                displayMessage(data.response, 'bot-message');

                // Xóa input
                document.getElementById('user-input').value = '';
            });

            function displayMessage(message, className) {
                const messageElement = document.createElement('div');
                messageElement.className = `message ${className}`;
                messageElement.innerText = message;
                document.getElementById('chat-container').appendChild(messageElement);
                document.getElementById('chat-container').scrollTop = document.getElementById('chat-container').scrollHeight; // Cuộn xuống cuối
            }
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)

# Route để nhận câu hỏi và trả về câu trả lời từ mô hình
@app.post("/ask/")
def ask_question(question: Question):
    messages = [
        {"role": "user", "content": question.question},
    ]
    response = model_handler.generate_response(messages)
    return {"response": response}

# Khởi tạo ngrok tunnel
ngrok.set_auth_token("2muEgcLI72W3vl8i28eJE5rrSSP_GJQ9nDAa4eQgCqZkCQ2f")
public_url = ngrok.connect(8000)
print("Ngrok Tunnel URL:", public_url)

# Chạy server FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)


In [3]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import contextlib
import os
import torch
from transformers import TextStreamer
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import logging

# Tắt logging từ Unsloth
logging.getLogger("unsloth").setLevel(logging.ERROR)

# Áp dụng nest_asyncio để khắc phục xung đột asyncio trong notebook
nest_asyncio.apply()

# Định nghĩa lớp LanguageModelHandler
class LanguageModelHandler:
    def __init__(self, model_path, max_seq_length=2048, load_in_4bit=True):
        self.max_seq_length = max_seq_length
        self.load_in_4bit = load_in_4bit

        # Redirect stdout để ẩn thông báo
        with open(os.devnull, "w") as devnull:
            with contextlib.redirect_stdout(devnull):
                self.model, self.tokenizer = self.load_model(model_path)

    def load_model(self, model_path):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_path,
            max_seq_length=self.max_seq_length,
            load_in_4bit=self.load_in_4bit
        )
        
        # Gọi phương thức for_inference
        FastLanguageModel.for_inference(model)

        tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
        return model, tokenizer

    def generate_response(self, messages, max_new_tokens=512, temperature=0.1, min_p=0.1):
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        outputs = self.model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            min_p=min_p
        )
        
        result = self.tokenizer.batch_decode(outputs)

        # Loại bỏ các token không cần thiết
        cleaned_result = result[0].replace("<|begin_of_text|>", "").replace("<|eot_id|>", "").replace("<|start_header_id|>", "").replace("<|end_header_id|>", "").strip()

        return cleaned_result

# Tạo đối tượng model_handler
model_handler = LanguageModelHandler("/kaggle/input/llm-llama3-2/Model")

# Khởi tạo FastAPI app
app = FastAPI()

# Mô hình dữ liệu cho câu hỏi từ người dùng
class Question(BaseModel):
    question: str

# Route để kiểm tra API hoạt động và trả về trang HTML
@app.get("/", response_class=HTMLResponse)
def read_root():
    # Tạo một file HTML đơn giản
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Chatbot</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                margin: 0;
                padding: 20px;
            }
            #chat-container {
                border: 1px solid #ccc;
                border-radius: 5px;
                padding: 10px;
                background-color: #fff;
                max-width: 600px;
                margin: 0 auto;
                height: 400px;
                overflow-y: auto;
            }
            #user-input {
                width: 100%;
                padding: 10px;
                box-sizing: border-box;
            }
            .message {
                margin: 10px 0;
            }
            .user-message {
                text-align: right;
                color: blue;
            }
            .bot-message {
                text-align: left;
                color: green;
            }
        </style>
    </head>
    <body>

        <h1>Chatbot</h1>
        <div id="chat-container"></div>
        <input type="text" id="user-input" placeholder="Nhập câu hỏi của bạn..."/>
        <button id="send-button">Gửi</button>

        <script>
            document.getElementById('send-button').addEventListener('click', async () => {
                const userInput = document.getElementById('user-input').value;
                if (!userInput) return;

                // Hiển thị câu hỏi của người dùng
                displayMessage(userInput, 'user-message');

                // Gửi yêu cầu đến API
                const response = await fetch('/ask/', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ question: userInput }),
                });

                const data = await response.json();

                // Hiển thị câu trả lời của bot
                displayMessage(data.response, 'bot-message');

                // Xóa input
                document.getElementById('user-input').value = '';
            });

            function displayMessage(message, className) {
                const messageElement = document.createElement('div');
                messageElement.className = `message ${className}`;
                messageElement.innerText = message;
                document.getElementById('chat-container').appendChild(messageElement);
                document.getElementById('chat-container').scrollTop = document.getElementById('chat-container').scrollHeight; // Cuộn xuống cuối
            }
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)

# Route để nhận câu hỏi và trả về câu trả lời từ mô hình
@app.post("/ask/")
def ask_question(question: Question):
    messages = [
        {"role": "user", "content": question.question},
    ]
    response = model_handler.generate_response(messages)
    return {"response": response}

# Khởi tạo ngrok tunnel
ngrok.set_auth_token("2muEgcLI72W3vl8i28eJE5rrSSP_GJQ9nDAa4eQgCqZkCQ2f")
public_url = ngrok.connect(8000)
print("Ngrok Tunnel URL:", public_url)

# Chạy server FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:     Started server process [31]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Ngrok Tunnel URL: NgrokTunnel: "https://4ef2-34-168-15-162.ngrok-free.app" -> "http://localhost:8000"
INFO:     58.186.75.18:0 - "GET / HTTP/1.1" 200 OK
INFO:     58.186.75.18:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


INFO:     58.186.75.18:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     58.186.75.18:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     58.186.75.18:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     58.186.75.18:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     58.186.75.18:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     58.186.75.18:0 - "POST /ask/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [31]


KeyboardInterrupt: 